In [17]:
from __future__ import print_function
from matplotlib import pyplot
pyplot.style.use('dark_background')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>table.dataframe {font-size:60%;line-height:100%; padding:0px; margin:0px;}</style>"))
display(HTML("<style>table.dataframe td {padding:3px; margin:0px;}</style>"))
display(HTML("<style>table.dataframe tr {padding:3px; margin:0px;}</style>"))
display(HTML("<style>table.dataframe th {padding:3px; margin:0px;}</style>"))
import yaml
#
from mimic3benchmark.mimic3csv import *
from mimic3benchmark.preprocessing import add_hcup_ccs_2015_groups, make_phenotype_label_matrix
from mimic3benchmark.util import *
from pandas.testing import assert_frame_equal
from OF_util import read_csv_file, write_csv_file, args, peek, read_and_cut, read_and_cut_multi_index

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
stays_sampled = read_csv_file('stays_all_drop_sampled.csv',root='./')
display(stays_sampled.sort_values('SUBJECT_ID'))
print(stays_sampled.ICUSTAY_ID.is_unique)
stays_sampled[stays_sampled.ICUSTAY_ID==212246]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LOS,Hos_LOS,Num_Prev_Hos_Adm,positive,negative
315,36,182104,280987,1.1096,8.281250,0,False,True
91,68,108329,272667,6.3983,14.547917,2,True,False
98,80,115385,256068,1.1901,4.986806,0,False,True
99,81,175016,222874,1.5137,1.756944,0,True,False
117,103,130744,212246,3.0237,7.734722,0,True,False
...,...,...,...,...,...,...,...,...
47883,99836,101018,200347,16.1103,56.879861,1,True,False
47884,99836,101018,240303,19.8529,56.879861,2,True,False
47900,99883,198523,293088,1.2756,6.600000,1,False,True
47901,99893,128349,208873,2.1240,6.679861,0,False,True


True


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,LOS,Hos_LOS,Num_Prev_Hos_Adm,positive,negative
117,103,130744,212246,3.0237,7.734722,0,True,False


In [6]:
num_samples = 1000
patients_cut = read_and_cut('PATIENTS.csv',col_to_cut='SUBJECT_ID', col_values=stays_sampled.SUBJECT_ID)
# check if subject id is unique:
print (patients_cut.SUBJECT_ID.is_unique)

write_csv_file(patients_cut, 'PATIENTS.csv')
# confirm written file will be loaded same
display(patients_cut.sort_values('SUBJECT_ID'))

True


,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
ROW_ID,,,,,,,
32,36,M,2061-08-17 00:00:00,NaN,NaN,NaN,0
61,68,F,2132-02-29 00:00:00,2174-02-11 00:00:00,NaN,2174-02-11 00:00:00,1
72,80,M,2104-12-13 00:00:00,NaN,NaN,NaN,0
73,81,M,2106-12-20 00:00:00,2192-01-12 00:00:00,2192-01-12 00:00:00,2192-01-12 00:00:00,1
94,103,F,2084-04-14 00:00:00,2144-09-01 00:00:00,2144-09-01 00:00:00,2144-09-01 00:00:00,1
...,...,...,...,...,...,...,...
46471,99822,M,2136-03-17 00:00:00,2197-07-08 00:00:00,2197-07-08 00:00:00,2197-07-08 00:00:00,1
46475,99836,F,2064-04-13 00:00:00,2116-07-23 00:00:00,NaN,2116-07-23 00:00:00,1
46489,99883,M,2058-02-04 00:00:00,NaN,NaN,NaN,0


In [8]:
peek('ICUSTAYS.csv')
stays_cut = read_and_cut('ICUSTAYS.csv', 'ICUSTAY_ID', stays_sampled.ICUSTAY_ID)
write_csv_file(stays_cut, 'ICUSTAYS.csv')
print (stays_cut.shape)

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE',
       'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID',
       'INTIME', 'OUTTIME', 'LOS'],
      dtype='object')

(3538, 11)

In [10]:
peek('ADMISSIONS.csv')
adm_cut = read_and_cut('ADMISSIONS.csv', 'HADM_ID', stays_sampled.HADM_ID)
write_csv_file(adm_cut, 'ADMISSIONS.csv')
print (adm_cut.shape)

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')

(3358, 18)

In [24]:
peek('TRANSFERS.csv')
transfer_cut = read_and_cut_multi_index('TRANSFERS.csv', stays_sampled[['SUBJECT_ID','HADM_ID','ICUSTAY_ID']])
write_csv_file(transfer_cut, 'TRANSFERS.csv')
print (transfer_cut.shape)
display(transfer_cut)

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'DBSOURCE',
       'EVENTTYPE', 'PREV_CAREUNIT', 'CURR_CAREUNIT', 'PREV_WARDID',
       'CURR_WARDID', 'INTIME', 'OUTTIME', 'LOS'],
      dtype='object')

(4636, 12)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
0,36,182104,280987.0,carevue,transfer,NaN,CSRU,49.0,12.0,2131-05-04 10:29:14,2131-05-05 13:07:03,26.63
1,68,108329,272667.0,carevue,transfer,NaN,MICU,45.0,23.0,2174-01-08 13:12:06,2174-01-14 22:45:42,153.56
2,109,155726,254406.0,metavision,admit,NaN,SICU,NaN,57.0,2142-08-13 04:04:27,2142-08-14 21:55:48,41.86
3,177,143120,255303.0,carevue,transfer,NaN,MICU,4.0,52.0,2125-12-02 15:49:45,2125-12-12 15:01:55,239.20
4,181,102631,246694.0,carevue,admit,NaN,CSRU,NaN,23.0,2153-10-12 09:52:00,2153-10-13 06:29:00,20.62
...,...,...,...,...,...,...,...,...,...,...,...,...
1232,94581,132584,275402.0,metavision,admit,NaN,SICU,NaN,33.0,2119-09-11 04:04:57,2119-09-11 04:14:26,0.16
1233,94581,132584,275402.0,metavision,transfer,SICU,SICU,33.0,57.0,2119-09-11 04:14:26,2119-09-12 17:28:16,37.23
1234,94581,132584,275402.0,metavision,transfer,SICU,MICU,57.0,52.0,2119-09-12 17:28:16,2119-09-14 21:44:54,52.28
1235,94597,178775,238794.0,metavision,admit,NaN,CCU,NaN,7.0,2169-11-09 01:07:40,2169-11-09 01:41:27,0.56


In [26]:
peek('DIAGNOSES_ICD.csv')
diagnoses_cut = read_and_cut_multi_index('DIAGNOSES_ICD.csv', stays_sampled[['SUBJECT_ID','HADM_ID']])
write_csv_file(diagnoses_cut, 'DIAGNOSES_ICD.csv')
print (diagnoses_cut.shape)
display(diagnoses_cut)

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,68,108329,1.0,042
1,68,108329,2.0,486
2,68,108329,3.0,4254
3,68,108329,4.0,2762
4,68,108329,5.0,5849
...,...,...,...,...
3911,96023,116385,6.0,1628
3912,96023,116385,7.0,5849
3913,96023,116385,8.0,73300
3914,96023,116385,9.0,7904


In [31]:
peek('PROCEDURES_ICD.csv')
procedures_cut = read_and_cut_multi_index('PROCEDURES_ICD.csv', stays_sampled[['SUBJECT_ID','HADM_ID']])
write_csv_file(procedures_cut, 'PROCEDURES_ICD.csv')
print (procedures_cut.shape)
display(procedures_cut)

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'], dtype='object')

(17282, 4)


,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,308,166606,1,3129
1,308,166606,2,9604
2,308,166606,3,9672
3,308,166606,4,9390
4,308,166606,5,9915
...,...,...,...,...
2886,43323,107385,6,3897
2887,43323,107385,7,3897
2888,43323,107385,8,370
2889,41035,102460,1,3228


In [29]:
peek('PRESCRIPTIONS.csv')
prescriptions_cut = read_and_cut_multi_index('PRESCRIPTIONS.csv', stays_sampled[['SUBJECT_ID','HADM_ID','ICUSTAY_ID']])
write_csv_file(prescriptions_cut, 'PRESCRIPTIONS.csv')
print (prescriptions_cut.shape)
display(prescriptions_cut)

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE',
       'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC',
       'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX',
       'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE'],
      dtype='object')

f:\75.python\venv\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(175601, 18)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,36,182104,280987.0,2131-05-04 00:00:00,2131-05-04 00:00:00,MAIN,Propofol,Propofol,Propofol,PROP100IG,016796,3.100300e+08,1000mg/100mL Vial,1000,mg,1,VIAL,IV DRIP
1,36,182104,280987.0,2131-05-04 00:00:00,2131-05-04 00:00:00,MAIN,Propofol,Propofol,Propofol,PROP500IG,016796,3.100300e+08,500mg/50mL Vial,500,mg,1,VIAL,IV DRIP
2,36,182104,280987.0,2131-05-04 00:00:00,2131-05-04 00:00:00,MAIN,Nitroglycerin,NaN,NaN,NITR20SYR,015361,7.414840e+07,20MG/50ML SYR,20,mg,1,SYR,IV DRIP
3,36,182104,280987.0,2131-05-04 00:00:00,2131-05-04 00:00:00,BASE,Syringe,NaN,NaN,SYR60ML,NaN,0.000000e+00,60mL Syringe,1,SYR,1,SYR,IV DRIP
4,36,182104,280987.0,2131-05-04 00:00:00,2131-05-04 00:00:00,BASE,Syringe,NaN,NaN,SYR60ML,NaN,0.000000e+00,60mL Syringe,1,SYR,1,SYR,IV DRIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,99995,137810,229633.0,2147-02-10 00:00:00,2147-02-11 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO40,008209,1.821162e+08,40mg Tablet,40,mg,1,TAB,PO/NG
2433,99995,137810,229633.0,2147-02-10 00:00:00,2147-02-11 00:00:00,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,6.332303e+10,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC
2434,99995,137810,229633.0,2147-02-11 00:00:00,2147-02-11 00:00:00,MAIN,Lisinopril,Lisinopril,Lisinopril,LISI10,000390,9.045809e+08,10mg Tablet,10,mg,1,TAB,PO/NG
2435,99995,137810,229633.0,2147-02-11 00:00:00,2147-02-11 00:00:00,MAIN,Ciprofloxacin HCl,Ciprofloxacin HCl,Ciprofloxacin HCl,CIPR250,009509,1.725311e+08,250mg Tab,250,mg,1,TAB,PO/NG


In [30]:
peek('CHARTEVENTS.csv')
chart_events_cut = read_and_cut_multi_index('CHARTEVENTS.csv', stays_sampled[['SUBJECT_ID','HADM_ID','ICUSTAY_ID']])
write_csv_file(chart_events_cut, 'CHARTEVENTS.csv')
print (chart_events_cut.shape)
display(chart_events_cut.head())

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME',
       'STORETIME', 'CGID', 'VALUE', 'VALUENUM', 'VALUEUOM', 'WARNING',
       'ERROR', 'RESULTSTATUS', 'STOPPED'],
      dtype='object')

f:\75.python\venv\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(20078130, 14)


,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,109,155726,254406.0,220180,2142-08-14 15:00:00,2142-08-14 15:02:00,17249.0,75,75.0,mmHg,0.0,0.0,NaN,NaN
1,109,155726,254406.0,220181,2142-08-14 15:00:00,2142-08-14 15:02:00,17249.0,91,91.0,mmHg,0.0,0.0,NaN,NaN
2,109,155726,254406.0,220210,2142-08-14 15:00:00,2142-08-14 15:01:00,17249.0,20,20.0,insp/min,0.0,0.0,NaN,NaN
3,109,155726,254406.0,220277,2142-08-14 15:00:00,2142-08-14 15:01:00,17249.0,95,95.0,%,0.0,0.0,NaN,NaN
4,109,155726,254406.0,226499,2142-08-14 15:48:00,2142-08-14 15:49:00,17249.0,1100,1100.0,mL,0.0,0.0,NaN,NaN
